In [3]:
import pandas as pd
import numpy as np

In [4]:
dfze=pd.read_csv('ready_house_price.csv')
dfze.head()

,Price,location,Area,Transaction,Furnishing,Bathroom
0,6000.0,67,500,1,2,1
1,13799.0,67,473,1,1,2
2,17500.0,67,779,1,2,2
3,18824.0,67,635,1,2,2
4,2538.0,67,550,1,2,1


In [238]:
x=dfze.drop(['Price'],axis=1).values
x

array([[  67,  500,    1,    2,    1],
       [  67,  473,    1,    1,    2],
       [  67,  779,    1,    2,    2],
       ...,
       [  80, 1705,    1,    1,    5],
       [  80,  895,    1,    1,    3],
       [  80, 1250,    1,    0,    3]], dtype=int64)

In [239]:
y=dfze['Price'].values
y

array([ 6000., 13799., 17500., ...,  4547.,  5135.,  4343.])

In [240]:
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [241]:
import torch
import torch.nn as nn
xte=torch.from_numpy(x).type(torch.float32).to(device)
yte=torch.from_numpy(y).type(torch.float32).to(device)

In [242]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(xte,yte,test_size=0.2)

In [243]:
# model defining
class HousePrice(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_stack=nn.Sequential(
            nn.Linear(in_features=5,out_features=10),
            nn.CELU(),
            nn.Linear(10,20),
            nn.CELU(),
            nn.Linear(20,30),
            nn.CELU(),
            nn.Linear(30,1)
        )
    def forward(self,x):
        return self.layer_stack(x)
model=HousePrice().to(device)

In [244]:
# loss function and optimizer
loss_fn=nn.L1Loss()
optimizer=torch.optim.Adam(params=model.parameters(),lr=0.0001)
xtrain=xtrain.to(device)
xtest=xtest.to(device)
ytrain=ytrain.to(device)
ytest=ytest.to(device)

In [253]:
# training and testing loop
from tqdm.auto import tqdm
epochs=100000


for epoch in tqdm(range(epochs)):
    # model training
    model.train()

    # ypred
    y_pred=model(xtrain)

    # loss cal
    loss=loss_fn(y_pred.squeeze(),ytrain)

    # OPTIMIZE ZERO GRAD
    optimizer.zero_grad()
    # loss backward
    loss.backward()

    # optimizer step
    optimizer.step()

    # model evaluating
    model.eval()

    with torch.inference_mode():
        test_predicted=model(xtest)

        test_loss=loss_fn(test_predicted.squeeze(),ytest)

        # printing test loss after n epochs
        if epoch%100==0:
            print(f"epoch: {epoch} | test_loss: {test_loss}")


  0%|          | 0/100000 [00:00<?, ?it/s]

epoch: 0 | test_loss: 2345.03515625
epoch: 100 | test_loss: 2344.957275390625
epoch: 200 | test_loss: 2344.93408203125
epoch: 300 | test_loss: 2344.8037109375
epoch: 400 | test_loss: 2344.753662109375
epoch: 500 | test_loss: 2344.573486328125
epoch: 600 | test_loss: 2344.588134765625
epoch: 700 | test_loss: 2344.591064453125
epoch: 800 | test_loss: 2344.4033203125
epoch: 900 | test_loss: 2344.310302734375
epoch: 1000 | test_loss: 2344.1796875
epoch: 1100 | test_loss: 2344.265625
epoch: 1200 | test_loss: 2344.187744140625
epoch: 1300 | test_loss: 2344.210205078125
epoch: 1400 | test_loss: 2343.835205078125
epoch: 1500 | test_loss: 2343.740234375
epoch: 1600 | test_loss: 2343.768310546875
epoch: 1700 | test_loss: 2343.493408203125
epoch: 1800 | test_loss: 2343.67578125
epoch: 1900 | test_loss: 2343.52490234375
epoch: 2000 | test_loss: 2343.327392578125
epoch: 2100 | test_loss: 2343.49462890625
epoch: 2200 | test_loss: 2343.118408203125
epoch: 2300 | test_loss: 2342.989013671875
epoch: 24

In [89]:
test_predicted[5]

tensor([4904.8149], device='cuda:0')

In [90]:
ytest[5]

tensor(4800., device='cuda:0')

In [92]:
from pathlib import Path
modelpath=Path('model')
modelpath.mkdir(parents=True,exist_ok=True)

In [94]:
model_name="housepredictonmodel"
modelsavepath=modelpath/model_name
torch.save(obj=model.state_dict(),f=modelsavepath)

In [247]:
map_tra=['New Property', 'Resale']
map_furnished=['Furnished', 'Semi-Furnished', 'Unfurnished']
map_loc=['agra',
 'ahmadnagar',
 'ahmedabad',
 'allahabad',
 'aurangabad',
 'badlapur',
 'bangalore',
 'belgaum',
 'bhiwadi',
 'bhiwandi',
 'bhopal',
 'bhubaneswar',
 'chandigarh',
 'chennai',
 'coimbatore',
 'dehradun',
 'durgapur',
 'ernakulam',
 'faridabad',
 'ghaziabad',
 'goa',
 'greater-noida',
 'guntur',
 'gurgaon',
 'guwahati',
 'gwalior',
 'haridwar',
 'hyderabad',
 'indore',
 'jabalpur',
 'jaipur',
 'jamshedpur',
 'jodhpur',
 'kalyan',
 'kanpur',
 'kochi',
 'kolkata',
 'kozhikode',
 'lucknow',
 'ludhiana',
 'madurai',
 'mangalore',
 'mohali',
 'mumbai',
 'mysore',
 'nagpur',
 'nashik',
 'navi-mumbai',
 'navsari',
 'nellore',
 'new-delhi',
 'noida',
 'palakkad',
 'palghar',
 'panchkula',
 'patna',
 'pondicherry',
 'pune',
 'raipur',
 'rajahmundry',
 'ranchi',
 'satara',
 'shimla',
 'siliguri',
 'solapur',
 'sonipat',
 'surat',
 'thane',
 'thrissur',
 'tirupati',
 'trichy',
 'trivandrum',
 'udaipur',
 'udupi',
 'vadodara',
 'vapi',
 'varanasi',
 'vijayawada',
 'visakhapatnam',
 'vrindavan',
 'zirakpur']

In [117]:
# preparing functions for user input 
# user will give ->	location , Area , Transaction ,	Furnishing , Bathroom
def predictingprice(location:int,Area:float,Transaction:int,Furnishing:int,Bathroom:int):
    list=[location,Area,Transaction,Furnishing,Bathroom]
    list=np.array(list)
    list=torch.from_numpy(list).type(torch.float32).to(device)
    model.eval()
    with torch.inference_mode():
        predicted=model(list).squeeze()
    predicted=predicted.cpu().detach().numpy()
    print(predicted)

In [252]:
predictingprice(67,500.0,1,2,2)

9594.51


In [231]:
mapping[67],map_tra[1],map_fur[2]

('thane', 'Resale', 'Unfurnished')

In [250]:
# mapping to index for prediction
loca,transaction,furnishing='thane','Resale','Unfurnished'
Area=500.0
Bathroom=2
index_loc = map_loc.index(loca)
index_trans = map_tra.index(transaction)
index_furnishing = map_fur.index(furnishing)

In [254]:
predictingprice(index_loc,Area,index_trans,index_furnishing,Bathroom)

8585.046
